In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import json
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate, cross_val_score
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import RobustScaler
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.width', 500)
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import MinMaxScaler


In [2]:
df= pd.read_csv('../input/fmworkdata/dflast4.csv')


In [3]:
positions = {
'GK': {'Hea': 1, 'Tec': 1, 'Ant': 3, 'Bra': 6, 'Cmp': 2, 'Cnt': 6, 'Dec': 10,
      'Ldr': 2, 'Pos': 5, 'Tea': 2, 'Vis': 1, 'Wor': 1, 'Acc': 6, 'Agi': 8,
      'Bal': 2, 'Jum': 1, 'Pac': 3, 'Sta': 1, 'Str': 4, 'Aer': 6, 'Cmd': 6,
      'Com': 5, 'Fir': 1, 'Han': 8, 'Kic': 5, '1v1': 4, 'Pas': 3, 'Ref': 8, 'Thr': 3},

'DRL': {'Cor': 1, 'Cro': 2, 'Dri': 1, 'Fin': 1, 'Fir': 3, 'Fre': 1, 'Hea': 2,
      'Lon': 1, 'L Th': 1, 'Mar': 3, 'Pas': 2, 'Pen': 1, 'Tck': 4, 'Tec': 2,
      'Ant': 3, 'Bra': 2, 'Cmp': 2, 'Cnt': 4, 'Dec': 7, 'Ldr': 1, 'OtB': 1,
      'Pos': 4, 'Tea': 2, 'Vis': 2, 'Wor': 2, 'Acc': 7, 'Agi': 6, 'Bal': 2,
      'Jum': 2, 'Pac': 5, 'Sta': 6, 'Str': 4},

'DC': {'Cor': 1, 'Cro': 1, 'Dri': 1, 'Fin': 1, 'Fir': 2, 'Fre': 1, 'Hea': 5,
      'Lon': 1, 'L Th': 1, 'Mar': 8, 'Pas': 2, 'Pen': 1, 'Tck': 5, 'Tec': 1,
      'Ant': 5, 'Bra': 2, 'Cmp': 2, 'Cnt': 4, 'Dec': 10, 'Ldr': 2, 'OtB': 1,
      'Pos': 8, 'Tea': 1, 'Vis': 1, 'Wor': 2, 'Acc': 6, 'Agi': 6, 'Bal': 2,
      'Jum': 6, 'Pac': 5, 'Sta': 3, 'Str': 6},

'WBRL': {'Cor': 1, 'Cro': 3, 'Dri': 2, 'Fin': 1, 'Fir': 3, 'Fre': 1, 'Hea': 1,
        'Lon': 1, 'L Th': 1, 'Mar': 2, 'Pas': 3, 'Pen': 1, 'Tck': 3, 'Tec': 3,
        'Ant': 3, 'Bra': 1, 'Cmp': 2, 'Cnt': 3, 'Dec': 5, 'Ldr': 1, 'OtB': 2,
        'Pos': 3, 'Tea': 2, 'Vis': 2, 'Wor': 2, 'Acc': 8, 'Agi': 5, 'Bal': 2,
        'Jum': 1, 'Pac': 6, 'Sta': 7, 'Str': 4},

'DM': {'Cor': 1, 'Cro': 1, 'Dri': 2, 'Fin': 2, 'Fir': 4, 'Fre': 1, 'Hea': 1,
      'Lon': 3, 'L Th': 1, 'Mar': 3, 'Pas': 4, 'Pen': 1, 'Tck': 7, 'Tec': 3,
      'Ant': 5, 'Bra': 1, 'Cmp': 2, 'Cnt': 3, 'Dec': 8, 'Ldr': 1, 'OtB': 1,
      'Pos': 5, 'Tea': 2, 'Vis': 4, 'Wor': 4, 'Acc': 6, 'Agi': 6, 'Bal': 2,
      'Jum': 1, 'Pac': 4, 'Sta': 4, 'Str': 5},

'MRL': {'Cor': 1, 'Cro': 5, 'Dri': 3, 'Fin': 2, 'Fir': 4, 'Fre': 1, 'Hea': 1,
       'Lon': 2, 'L Th': 1, 'Mar': 1, 'Pas': 3, 'Pen': 1, 'Tck': 2, 'Tec': 4,
       'Ant': 3, 'Bra': 1, 'Cmp': 3, 'Cnt': 2, 'Dec': 5, 'Ldr': 1, 'OtB': 2,
       'Pos': 1, 'Tea': 2, 'Vis': 3, 'Wor': 3, 'Acc': 8, 'Agi': 6, 'Bal': 2,
       'Jum': 1, 'Pac': 6, 'Sta': 5, 'Str': 3},

'MC': {'Cor': 1, 'Cro': 1, 'Dri': 2, 'Fin': 2, 'Fir': 6, 'Fre': 1, 'Hea': 1,
      'Lon': 3, 'L Th': 1, 'Mar': 3, 'Pas': 6, 'Pen': 1, 'Tck': 3, 'Tec': 4,
      'Ant': 3, 'Bra': 1, 'Cmp': 3, 'Cnt': 2, 'Dec': 7, 'Ldr': 1, 'OtB': 3,
      'Pos': 3, 'Tea': 2, 'Vis': 6, 'Wor': 3, 'Acc': 6, 'Agi': 6, 'Bal': 2,
      'Jum': 1, 'Pac': 5, 'Sta': 6, 'Str': 4},

'AMRL': {'Cor': 1, 'Cro': 5, 'Dri': 5, 'Fin': 2, 'Fir': 5, 'Fre': 1, 'Hea': 1,
        'Lon': 2, 'L Th': 1, 'Mar': 1, 'Pas': 2, 'Pen': 1, 'Tck': 2, 'Tec': 4,
        'Ant': 3, 'Bra': 1, 'Cmp': 3, 'Cnt': 2, 'Dec': 5, 'Ldr': 1, 'OtB': 2,
        'Pos': 1, 'Tea': 2, 'Vis': 3, 'Wor': 3, 'Acc': 10, 'Agi': 6, 'Bal': 2,
        'Jum': 1, 'Pac': 10, 'Sta': 7, 'Str': 3},

'AMC': {'Cor': 1, 'Cro': 1, 'Dri': 3, 'Fin': 3, 'Fir': 5, 'Fre': 1, 'Hea': 1,
       'Lon': 3, 'L Th': 1, 'Mar': 1, 'Pas': 4, 'Pen': 1, 'Tck': 2, 'Tec': 5,
       'Ant': 3, 'Bra': 1, 'Cmp': 3, 'Cnt': 2, 'Dec': 6, 'Ldr': 1, 'OtB': 3,
       'Pos': 2, 'Tea': 2, 'Vis': 6, 'Wor': 3, 'Acc': 9, 'Agi': 6, 'Bal': 2,
       'Jum': 1, 'Pac': 7, 'Sta': 6, 'Str': 3},

'SC': {'Cor': 1, 'Cro': 2, 'Dri': 5, 'Fin': 8, 'Fir': 6, 'Fre': 1, 'Hea': 6,
      'Lon': 2, 'L Th': 1, 'Mar': 1, 'Pas': 2, 'Pen': 1, 'Tck': 1, 'Tec': 4,
      'Ant': 5, 'Bra': 1, 'Cmp': 6, 'Cnt': 2, 'Dec': 5, 'Ldr': 1, 'OtB': 6,
      'Pos': 2, 'Tea': 1, 'Vis': 2, 'Wor': 2, 'Acc': 10, 'Agi': 6, 'Bal': 2,
      'Jum': 5, 'Pac': 7, 'Sta': 6, 'Str': 6},
}


In [4]:
positions_20 = {}
positions_21 = {}

for key, value in positions.items():
  positions_20[key] = {key + '_20':value[key] for key in value.keys()}
  positions_21[key] = {key + '_21':value[key] for key in value.keys()}


In [5]:
scaler = MinMaxScaler((0,100))
for position in list(positions.keys()):
  for suff, pos_dict in {'_22':positions, '_21':positions_21, '_20':positions_20}.items():
    df[position + suff] = 0
    for key, value in pos_dict[position].items():
      df[position + suff] += df[key] * value

  df[position] = df[position + '_22'] * 0.55 + df[position + '_21'] * 0.35 + df[position + '_20'] * 0.1
  df = df.drop([position + '_20', position + '_21', position + '_22'], axis=1)
  df[position] = scaler.fit_transform(df[[position]])

In [6]:
df.head()

,UID,Wage,Name,Nat,Based,Club,Position,Preferred Foot,Right Foot,Left Foot,Height,Weight,Age,Transfer Value,AT Apps,Caps,Yth Apps,Rc Injury,Agg,Jum,Pun,Nat .1,Vis,L Th,Lon,OtB,Tck,Tec,Tea,Cmp,Fre,Ref,Pos,Pen,Pas,Fla,Ant,Cro,Mar,Ldr,Cor,Cnt,Det,Dec,Hea,Fir,Com,Acc,Pac,Aer,Str,Thr,Han,Ecc,Dri,Bal,Kic,Sta,Agi,Wor,Bra,Cmd,Fin,1v1,TRO,Value,Agg_21,Jum_21,Pun_21,Nat .1_21,Vis_21,L Th_21,Lon_21,OtB_21,Tck_21,Tec_21,Tea_21,Cmp_21,Fre_21,Ref_21,Pos_21,Pen_21,Pas_21,Fla_21,Ant_21,Cro_21,Mar_21,Ldr_21,Cor_21,Cnt_21,Det_21,Dec_21,Hea_21,Fir_21,Com_21,Acc_21,Pac_21,Aer_21,Str_21,Thr_21,Han_21,Ecc_21,Dri_21,Bal_21,Kic_21,Sta_21,Agi_21,Wor_21,Bra_21,Cmd_21,Fin_21,1v1_21,TRO_21,Agg_20,Jum_20,Pun_20,Nat .1_20,Vis_20,L Th_20,Lon_20,OtB_20,Tck_20,Tec_20,Tea_20,Cmp_20,Fre_20,Ref_20,Pos_20,Pen_20,Pas_20,Fla_20,Ant_20,Cro_20,Mar_20,Ldr_20,Cor_20,Cnt_20,Det_20,Dec_20,Hea_20,Fir_20,Com_20,Acc_20,Pac_20,Aer_20,Str_20,Thr_20,Han_20,Ecc_20,Dri_20,Bal_20,Kic_20,Sta_20,Agi_20,Wor_20,Bra_20,Cmd_20,Fin_20,1v1_20,TRO_20,Role,GK,DRL,DC,WBRL,DM,MRL,MC,AMRL,AMC,SC,AT Gls
0,7458500,73800000.000,Lionel Messi,ARG,France (Ligue 1),PSG,AMC,Left,Fairly Strong,Very Strong,169 cm,67 kg,34,95360630.000,552.000,1,18,0,7,6,3,14,20,4,17,14,7,20,13,17,19,3,5,17,19,20,18,15,4,14,15,11,20,19,10,19,2,16,15,1,9,1,2,3,20,18,1,12,16,7,10,2,18,3,3,53000000.000,7,6,2,14,20,4,17,16,7,20,14,18,19,3,5,17,20,20,19,15,4,14,15,13,20,20,10,19,1,17,15,2,9,2,1,1,20,19,1,13,17,7,10,3,20,2,1,7,6,2,14,20,4,17,16,7,20,14,18,19,1,5,17,20,20,19,15,4,14,15,13,20,20,10,19,4,18,15,2,9,1,3,1,20,19,1,13,19,7,10,3,20,3,1,Attacking Midfielder (Attack),49.099,82.517,71.762,88.006,85.847,95.087,93.198,95.377,97.035,92.669,485.000
1,18004457,20676000.000,Kevin De Bruyne,BEL,England (Premier League),Manchester City,MC,Either,Very Strong,Strong,181 cm,68 kg,30,248000000.000,362.000,1,2,0,12,10,2,16,20,8,16,14,9,18,14,15,17,2,9,16,18,16,13,19,7,12,14,15,15,18,6,16,3,15,14,2,13,3,3,2,15,14,1,17,13,15,12,3,16,3,2,94000000.000,12,10,1,16,20,8,16,15,9,18,14,15,17,2,9,16,18,16,13,19,7,12,15,15,15,18,6,16,2,15,14,1,13,1,1,1,15,15,3,17,13,15,12,3,16,1,3,12,9,3,16,20,8,16,15,9,17,14,15,15,3,9,12,19,15,13,19,7,12,15,15,15,18,6,16,3,15,14,3,13,2,3,1,15,14,4,17,13,15,14,2,16,2,1,Mezzala (Attack),48.809,83.796,74.105,88.264,85.635,92.300,90.995,92.079,92.712,86.287,81.000
2,735216,29196000.000,Cristiano Ronaldo,POR,England (Premier League),Man Utd,SC,Either,Very Strong,Strong,187 cm,84 kg,36,34269360.000,613.000,1,10,0,6,17,3,20,12,5,15,18,2,16,6,15,10,1,2,19,13,17,19,13,3,18,10,14,20,13,19,15,1,13,13,4,15,2,3,2,12,14,4,13,11,7,12,2,19,3,1,10500000.000,6,17,1,20,14,3,19,18,8,17,10,14,13,1,6,20,14,17,18,15,3,19,13,14,20,18,19,17,1,15,16,2,16,3,1,1,15,14,2,15,13,10,16,4,19,3,2,6,17,2,20,14,3,19,18,8,17,10,14,15,3,6,20,15,18,18,15,4,17,13,16,20,18,19,17,1,15,16,3,16,3,2,1,15,14,2,15,13,10,16,4,19,2,2,Poacher (Attack),45.451,74.728,71.948,77.928,73.560,82.318,79.304,82.616,82.811,90.639,479.000
3,85139014,25104000.000,Kylian Mbappé,FRA,France (Ligue 1),PSG,SC,Right,Very Strong,Reasonable,178 cm,73 kg,22,249000000.000,148.000,1,-,0,6,8,1,15,15,4,13,19,4,17,10,18,12,3,3,15,14,18,17,13,4,10,12,14,15,15,7,17,3,20,20,3,11,2,1,2,18,15,1,14,16,10,12,2,17,2,3,90000000.000,6,8,3,15,13,2,12,18,2,17,10,18,10,4,2,15,12,18,17,13,2,10,10,13,14,14,7,16,3,20,20,1,11,3,3,1,18,14,2,14,16,10,12,2,17,3,3,6,8,2,15,13,2,12,18,2,17,10,18,10,3,2,15,13,18,17,13,2,10,10,13,14,16,7,16,2,20,20,2,11,2,1,2,18,14,3,14,16,10,12,1,17,3,3,Advanced Forward (Attack),46.613,76.061,64.153,83.041,74.595,88.743,82.715,92.111,89.600,89.649,107.000
4,98028755,13680000.000,Mohamed Salah,EGY,England (Premier League),Liverpool,AMRL,Left,Weak,Very Strong,175 cm,72 kg,29,136000000.000,325.000,1,11,0,10,8,3,17,17,6,13,18,7,17,15,15,12,2,9,17,15,15,16,14,7,10,12,15,16,15,10,16,1,18,18,2,15,2,3,3,17,18,3,15,16,15,11,1,18,1,3,84000000.000,10,8,2,17,17,6,13,18,7,16,14,14,12,1,9,15,15,15,16,14,7,10,13,14,16,14,9,15,3,18,18,3,15,2,1,4,16,18,1,14,16,15,11,3,18,3,2,10,7,1,16,16,4

In [7]:
df[df["UID"]==777777777]

,UID,Wage,Name,Nat,Based,Club,Position,Preferred Foot,Right Foot,Left Foot,Height,Weight,Age,Transfer Value,AT Apps,Caps,Yth Apps,Rc Injury,Agg,Jum,Pun,Nat .1,Vis,L Th,Lon,OtB,Tck,Tec,Tea,Cmp,Fre,Ref,Pos,Pen,Pas,Fla,Ant,Cro,Mar,Ldr,Cor,Cnt,Det,Dec,Hea,Fir,Com,Acc,Pac,Aer,Str,Thr,Han,Ecc,Dri,Bal,Kic,Sta,Agi,Wor,Bra,Cmd,Fin,1v1,TRO,Value,Agg_21,Jum_21,Pun_21,Nat .1_21,Vis_21,L Th_21,Lon_21,OtB_21,Tck_21,Tec_21,Tea_21,Cmp_21,Fre_21,Ref_21,Pos_21,Pen_21,Pas_21,Fla_21,Ant_21,Cro_21,Mar_21,Ldr_21,Cor_21,Cnt_21,Det_21,Dec_21,Hea_21,Fir_21,Com_21,Acc_21,Pac_21,Aer_21,Str_21,Thr_21,Han_21,Ecc_21,Dri_21,Bal_21,Kic_21,Sta_21,Agi_21,Wor_21,Bra_21,Cmd_21,Fin_21,1v1_21,TRO_21,Agg_20,Jum_20,Pun_20,Nat .1_20,Vis_20,L Th_20,Lon_20,OtB_20,Tck_20,Tec_20,Tea_20,Cmp_20,Fre_20,Ref_20,Pos_20,Pen_20,Pas_20,Fla_20,Ant_20,Cro_20,Mar_20,Ldr_20,Cor_20,Cnt_20,Det_20,Dec_20,Hea_20,Fir_20,Com_20,Acc_20,Pac_20,Aer_20,Str_20,Thr_20,Han_20,Ecc_20,Dri_20,Bal_20,Kic_20,Sta_20,Agi_20,Wor_20,Bra_20,Cmd_20,Fin_20,1v1_20,TRO_20,Role,GK,DRL,DC,WBRL,DM,MRL,MC,AMRL,AMC,SC,AT Gls
105644,777777777,80000000.000,Vahit Keskin,TUR,TURKEY (Ligue 1),VBO,AMC,Left,Fairly Strong,Very Strong,169 cm,67 kg,34,250000000.000,552.000,1,18,0,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16.000,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,Attacking Midfielder (Attack),100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,485.000


In [8]:
df.to_csv("fmmevki.csv",index=False)